In [1]:
import os
import sys
import ast
import json
import spacy
import pandas as pd
from collections import Counter

In [2]:
def process_bar(num, total):
    rate = float(num)/total
    ratenum = int(100*rate)
    r = '\r[{}{}]{}%'.format('*'*ratenum,' '*(100-ratenum), ratenum)
    sys.stdout.write(r)
    sys.stdout.flush()

In [3]:
def named_entity_info_freq(text_fr):
    nlp_model_fr = spacy.load("fr_core_news_lg")
    nlp_model_fr.add_pipe("entityfishing", config={"language": "fr"})
    doc_fr = nlp_model_fr(text_fr)

    ents = []
    ents_info = {}
    for ent in doc_fr.ents:
        ents.append(ent.text)
        ents_info[ent.text] = {'label': ent.label_,
                               'kb_qid': ent._.kb_qid,
                               'url_wikidata': ent._.url_wikidata}

    return ents_info, ents

In [30]:
def get_info_freq(path, file_list):
    year = []
    month = []
    page = []
    ents_info = []
    ents_freq = []
    x = 0
    n = len(file_list) 
    for file_name in file_list:
        f = file_name.split('.')[0]
        ymp = f.split('-')
        if ymp[0] == '':
            continue
        year.append(int(ymp[0]))
        month.append(int(ymp[1]))
        page.append(int(ymp[2]))

        with open(os.path.join(path, file_name)+'.txt') as f:
            text_fr = f.read()
        info, ents = named_entity_info_freq(text_fr)
        ents_info.append(info)
        freq = dict(Counter(ents))
        ents_freq.append(freq)

        x += 1
        process_bar(x+1, n)

        data = {'ents_info': info, 'ents_freq': freq, 'year': ymp[0], 'month': ymp[1], 'page': ymp[2]}
        json_data = json.dumps(data)
        f2 = open('/Users/DXY/Documents/Project/entity_json/'+ str(ymp[0]) + '-' + str(ymp[1]) + '-' + str(ymp[2]) + '.json', 'w')
        f2.write(json_data)
        f2.close()
            
    return ents_info, ents_freq, year, month, page

In [20]:
path1 = '/Users/DXY/Documents/Project/modified_text'
g1 = os.walk(path1)
file_name_list1 = []
for path, dir_list, file_list in g1:
    for file_name in file_list:
        file_name_list1.append(file_name.split('.')[0])

In [21]:
len(file_name_list1)

7950

In [23]:
path2 = '/Users/DXY/Documents/Project/entity_json/'
g1 = os.walk(path2)
file_name_list2 = []
for path, dir_list, file_list in g1:
    for file_name in file_list:
        file_name_list2.append(file_name.split('.')[0])

In [26]:
len(file_name_list2)

6054

In [27]:
list_run = [ fn for fn in file_name_list1 if fn not in file_name_list2 ] # b中有而a中没有的

In [28]:
len(list_run)

1896

In [31]:
ents_info, ents_freq, year, month, page = get_info_freq(path1, list_run)

[****************************************************************************************************]100%